# Variação Paramétrica do LVQ


# Passo 0: Preparando o ambiente

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

In [2]:
DATASET_ORIGINAL = "../datasets/data_normalizada.csv"
DATASET_SMOTE = "../datasets/data_smote.csv"
DATASET_UNDER = "../datasets/data_under.csv"
DATASET_TESTE = "../datasets/data_test.csv"

In [3]:
df: pd.DataFrame = pd.read_csv(DATASET_ORIGINAL)
df_smote: pd.DataFrame = pd.read_csv(DATASET_SMOTE)
df_under: pd.DataFrame = pd.read_csv(DATASET_UNDER)
df_test: pd.DataFrame = pd.read_csv(DATASET_TESTE)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 94 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Bankrupt?                                                6819 non-null   int64  
 1   ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2   ROA(A) before interest and % after tax                   6819 non-null   float64
 3   ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4   Operating Gross Margin                                   6819 non-null   float64
 5   Realized Sales Gross Margin                              6819 non-null   float64
 6   Operating Profit Rate                                    6819 non-null   float64
 7   Pre-tax net Interest Rate                                6819 non-null   float64
 8   After-tax net Interest Rate 

# Passo 1: Definindo o Algorítmo do LVQ

In [5]:
class LVQ:
    def __init__(self, learning_rate=0.1, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs

    def initialize_prototypes(self, X):
        self.prototypes = [X[0], X[1]]

    def find_closest_prototype(self, x):
        distances = np.linalg.norm(self.prototypes - x, axis=1)
        return np.argmin(distances)

    def update_prototypes(self, x, y, closest_prototype):
        if y == closest_prototype:
            self.prototypes[y] += self.learning_rate * (x - self.prototypes[y])
        else:
            self.prototypes[y] -= self.learning_rate * (x - self.prototypes[y])

    def fit(self, X, y):
        X_normalized = X / np.linalg.norm(X, axis=1)[:, np.newaxis]  # Normalize input data
        self.initialize_prototypes(X_normalized)
        for epoch in range(self.epochs):
            for x, label in zip(X_normalized, y):
                closest_prototype = self.find_closest_prototype(x)
                self.update_prototypes(x, label, closest_prototype)
            self.learning_rate *= 0.9  # Decay learning rate

    def predict(self, X):
        X_normalized = X / np.linalg.norm(X, axis=1)[:, np.newaxis]  # Normalize input data
        y_pred = []
        for x in X_normalized:
            closest_prototype = self.find_closest_prototype(x)
            y_pred.append(closest_prototype)
        return np.array(y_pred)
    
    def get_f1(self, X_test, y_test):
        return f1_score(y_test, self.predict(X_test))
    
    def evaluate(self, X_test, y_test):
        print(classification_report(y_test, self.predict(X_test)))

# Passo 2: Definição do Campo de busca

Aqui é definido os parâmetros que serão testados pelo modelo. Para o LVQ temos dois parâmetros: num_epochs e alpha. Segue abaixo uma breve explicação dos parâmetros testados:

 - num_epochs: número de épocas no treinamento
 - lr: taxa usada na atualização dos pesos

In [6]:
params = {
    "num_epochs": [3, 5, 10, 15, 20, 40, 80, 100],
    "lr": [1e-3, 1e-5, 1e-5]
}

# Passo 3: Definindo a Busca por Parâmetros

In [7]:
def LVQSearch(search_params: list, X_train: np.ndarray, X_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> list:
    best_model = None
    best_f1 = 0
    for lr in search_params["lr"]:
        for num_epochs in search_params["num_epochs"]:
            lvq_model = LVQ(learning_rate=lr, epochs=num_epochs)
            
            print(f"Buscando com lr {lr} e num_epochs {num_epochs}")
            lvq_model.fit(X_train, y_train)
            
            f1 = lvq_model.get_f1(X_test, y_test)
            print(f"f1: {f1}")
            if f1 > best_f1:
                best_f1 = f1
                print(f"Atualizando o melhor modelo para com lr {lr} e num_epochs {num_epochs} com f1 = {f1}")
                best_model = lvq_model
    best_model.evaluate(X_test, y_test)
    return best_model

# Passo 4: Realizando a busca para o dataset normalizado

In [8]:
X, y = df.iloc[:, 1:], df.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [9]:
best = LVQSearch(params, X_train.values, X_test.values, y_train.values, y_test.values)

Buscando com lr 0.001 e num_epochs 3
f1: 0.06456190138346932
Atualizando o melhor modelo para com lr 0.001 e num_epochs 3 com f1 = 0.06456190138346932
Buscando com lr 0.001 e num_epochs 5
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 10
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 15
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 20
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 40
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 80
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 100
f1: 0.06456190138346932
Buscando com lr 1e-05 e num_epochs 3
f1: 0.11538461538461539
Atualizando o melhor modelo para com lr 1e-05 e num_epochs 3 com f1 = 0.11538461538461539
Buscando com lr 1e-05 e num_epochs 5
f1: 0.11421319796954314
Buscando com lr 1e-05 e num_epochs 10
f1: 0.11117974058060531
Buscando com lr 1e-05 e num_epochs 15
f1: 0.10935601458080195
Buscando com lr 1e-05 e num_epochs 20
f1: 0.10849909584086799
Buscando com lr 1

O melhor modelo foi o com o macro f1 = 0.38, ainda é um valor muito baixo, vamos diminuir o lr, o número de épocas não fez diferença.

In [10]:
params_ = {
    "num_epochs": [3, 5, 10, 15, 20, 40, 80, 100],
    "lr": [1e-5, 1e-6, 1e-7, 1e-8]
}

In [11]:
best = LVQSearch(params_, X_train.values, X_test.values, y_train.values, y_test.values)

Buscando com lr 1e-05 e num_epochs 3
f1: 0.11538461538461539
Atualizando o melhor modelo para com lr 1e-05 e num_epochs 3 com f1 = 0.11538461538461539
Buscando com lr 1e-05 e num_epochs 5
f1: 0.11421319796954314
Buscando com lr 1e-05 e num_epochs 10
f1: 0.11117974058060531
Buscando com lr 1e-05 e num_epochs 15
f1: 0.10935601458080195
Buscando com lr 1e-05 e num_epochs 20
f1: 0.10849909584086799
Buscando com lr 1e-05 e num_epochs 40
f1: 0.10794780545670225
Buscando com lr 1e-05 e num_epochs 80
f1: 0.1076923076923077
Buscando com lr 1e-05 e num_epochs 100
f1: 0.1076923076923077
Buscando com lr 1e-06 e num_epochs 3
f1: 0.1167979002624672
Atualizando o melhor modelo para com lr 1e-06 e num_epochs 3 com f1 = 0.1167979002624672
Buscando com lr 1e-06 e num_epochs 5
f1: 0.11649214659685864
Buscando com lr 1e-06 e num_epochs 10
f1: 0.11641595814257685
Buscando com lr 1e-06 e num_epochs 15
f1: 0.116263879817113
Buscando com lr 1e-06 e num_epochs 20
f1: 0.11618798955613577
Buscando com lr 1e-06 e

O melhor modelo tem lr 1e-07 e num_epochs 3 com f1 = 0.11710526315789474

# Passo 5: Realizando a busca para o dataset smote

In [12]:
X_train, y_train = df_smote.iloc[:, 1:], df_smote.iloc[:, 0]
X_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

In [13]:
best = LVQSearch(params, X_train.values, X_test.values, y_train.values, y_test.values)

Buscando com lr 0.001 e num_epochs 3
f1: 0.06456190138346932
Atualizando o melhor modelo para com lr 0.001 e num_epochs 3 com f1 = 0.06456190138346932
Buscando com lr 0.001 e num_epochs 5
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 10
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 15
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 20
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 40
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 80
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 100
f1: 0.06456190138346932
Buscando com lr 1e-05 e num_epochs 3
f1: 0.10982306284319707
Atualizando o melhor modelo para com lr 1e-05 e num_epochs 3 com f1 = 0.10982306284319707
Buscando com lr 1e-05 e num_epochs 5
f1: 0.10637054354178843
Buscando com lr 1e-05 e num_epochs 10
f1: 0.09994508511806699
Buscando com lr 1e-05 e num_epochs 15
f1: 0.09665427509293681
Buscando com lr 1e-05 e num_epochs 20
f1: 0.0948905109489051
Buscando com lr 1e

O melhor modelo foi o com o macro f1 = 0.36, ainda é um valor muito baixo, vamos diminuir o lr, o número de épocas não fez diferença.

In [14]:
params_ = {
    "num_epochs": [3, 5, 10, 15, 20, 40, 80, 100],
    "lr": [1e-5, 1e-6, 1e-7, 1e-8]
}

In [15]:
best = LVQSearch(params_, X_train.values, X_test.values, y_train.values, y_test.values)

Buscando com lr 1e-05 e num_epochs 3
f1: 0.10982306284319707
Atualizando o melhor modelo para com lr 1e-05 e num_epochs 3 com f1 = 0.10982306284319707
Buscando com lr 1e-05 e num_epochs 5
f1: 0.10637054354178843
Buscando com lr 1e-05 e num_epochs 10
f1: 0.09994508511806699
Buscando com lr 1e-05 e num_epochs 15
f1: 0.09665427509293681
Buscando com lr 1e-05 e num_epochs 20
f1: 0.0948905109489051
Buscando com lr 1e-05 e num_epochs 40
f1: 0.09280979092299847
Buscando com lr 1e-05 e num_epochs 80
f1: 0.09238578680203045
Buscando com lr 1e-05 e num_epochs 100
f1: 0.09238578680203045
Buscando com lr 1e-06 e num_epochs 3
f1: 0.11611219830397912
Atualizando o melhor modelo para com lr 1e-06 e num_epochs 3 com f1 = 0.11611219830397912
Buscando com lr 1e-06 e num_epochs 5
f1: 0.11550940947436729
Buscando com lr 1e-06 e num_epochs 10
f1: 0.1159047005795235
Buscando com lr 1e-06 e num_epochs 15
f1: 0.11538461538461539
Buscando com lr 1e-06 e num_epochs 20
f1: 0.11516314779270634
Buscando com lr 1e-

O melhor modelo tem lr 1e-07 e num_epochs 3 com f1 = 0.11710526315789474

# Passo 6: Realizando a busca para o dataset under

In [16]:
X_train, y_train = df_under.iloc[:, 1:], df_under.iloc[:, 0]
X_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

In [17]:
best = LVQSearch(params, X_train.values, X_test.values, y_train.values, y_test.values)

Buscando com lr 0.001 e num_epochs 3
f1: 0.08351409978308026
Atualizando o melhor modelo para com lr 0.001 e num_epochs 3 com f1 = 0.08351409978308026
Buscando com lr 0.001 e num_epochs 5
f1: 0.07382828701783492
Buscando com lr 0.001 e num_epochs 10
f1: 0.06532663316582915
Buscando com lr 0.001 e num_epochs 15
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 20
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 40
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 80
f1: 0.06456190138346932
Buscando com lr 0.001 e num_epochs 100
f1: 0.06456190138346932
Buscando com lr 1e-05 e num_epochs 3
f1: 0.0826686004350979
Buscando com lr 1e-05 e num_epochs 5
f1: 0.08260869565217391
Buscando com lr 1e-05 e num_epochs 10
f1: 0.08254887762490949
Buscando com lr 1e-05 e num_epochs 15
f1: 0.08254887762490949
Buscando com lr 1e-05 e num_epochs 20
f1: 0.0824891461649783
Buscando com lr 1e-05 e num_epochs 40
f1: 0.0824295010845987
Buscando com lr 1e-05 e num_epochs 80
f1: 0.0824

O melhor modelo foi o com o macro f1 = 0.31, ainda é um valor muito baixo, vamos diminuir o lr, o número de épocas não fez diferença.

In [18]:
params_ = {
    "num_epochs": [3, 5, 10, 15, 20, 40, 80, 100],
    "lr": [1e-5, 1e-6, 1e-7, 1e-8]
}

In [19]:
best = LVQSearch(params_, X_train.values, X_test.values, y_train.values, y_test.values)

Buscando com lr 1e-05 e num_epochs 3
f1: 0.0826686004350979
Atualizando o melhor modelo para com lr 1e-05 e num_epochs 3 com f1 = 0.0826686004350979
Buscando com lr 1e-05 e num_epochs 5
f1: 0.08260869565217391
Buscando com lr 1e-05 e num_epochs 10
f1: 0.08254887762490949
Buscando com lr 1e-05 e num_epochs 15
f1: 0.08254887762490949
Buscando com lr 1e-05 e num_epochs 20
f1: 0.0824891461649783
Buscando com lr 1e-05 e num_epochs 40
f1: 0.0824295010845987
Buscando com lr 1e-05 e num_epochs 80
f1: 0.0824295010845987
Buscando com lr 1e-05 e num_epochs 100
f1: 0.0824295010845987
Buscando com lr 1e-06 e num_epochs 3
f1: 0.08272859216255443
Atualizando o melhor modelo para com lr 1e-06 e num_epochs 3 com f1 = 0.08272859216255443
Buscando com lr 1e-06 e num_epochs 5
f1: 0.08272859216255443
Buscando com lr 1e-06 e num_epochs 10
f1: 0.08272859216255443
Buscando com lr 1e-06 e num_epochs 15
f1: 0.08272859216255443
Buscando com lr 1e-06 e num_epochs 20
f1: 0.08272859216255443
Buscando com lr 1e-06 e

O melhor modelo tem lr 1e-07 e num_epochs 3 com f1 = 0.08272859216255443